In [1]:
import os

os.listdir ("./data")

['First and Future Data Manual 12062018.docx',
 'game_data.csv',
 'NGS-2016-post.csv',
 'NGS-2016-pre.csv',
 'NGS-2016-reg-wk1-6.csv',
 'NGS-2016-reg-wk13-17.csv',
 'NGS-2016-reg-wk7-12.csv',
 'NGS-2017-post.csv',
 'NGS-2017-pre.csv',
 'NGS-2017-reg-wk1-6.csv',
 'NGS-2017-reg-wk13-17.csv',
 'NGS-2017-reg-wk7-12.csv',
 'player_punt_data.csv',
 'play_information.csv',
 'play_player_role_data.csv',
 'video_footage-control.csv',
 'video_footage-injury.csv',
 'video_review.csv',
 '~$rst and Future Data Manual 12062018.docx']

In [2]:
import pandas as pd

df = pd.read_csv ("./data/NGS-2017-reg-wk1-6.csv", parse_dates = ['Time'])

df.sort_values (['GameKey', 'PlayID', 'GSISID', 'Time'], inplace = True)

df.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event
5144770,2017,399,1336,23449.0,2017-09-08 01:31:11.000,41.560001,54.279999,0.13,78.879997,192.050003,NaN
5144771,2017,399,1336,23449.0,2017-09-08 01:31:11.100,41.540001,54.139999,0.14,77.760002,190.759995,NaN
5144772,2017,399,1336,23449.0,2017-09-08 01:31:11.200,41.509998,54.000000,0.14,76.459999,189.559998,NaN
5144773,2017,399,1336,23449.0,2017-09-08 01:31:11.300,41.490002,53.860001,0.14,75.800003,188.449997,NaN
5144320,2017,399,1336,23449.0,2017-09-08 01:31:11.400,41.470001,53.709999,0.15,75.769997,187.589996,NaN


In [3]:
groupe = df.groupby (['Season_Year','GameKey', 'PlayID', 'GSISID'])["Time","Event"].count()
groupe.columns = ['Time_Player', 'Nb_event_played']
groupe.reset_index(inplace=True)
groupe.head(5)

Time_Player  Nb_event_played
Season_Year GameKey PlayID GSISID                               
2017        399     1336   23449.0           29                1
                           23534.0          205                6
                           26035.0          205                6
                           27010.0          205                6
                           27442.0          205                6

In [5]:
groupy = groupe.groupby(["Season_Year","GameKey","PlayID"])['Time_Player',"Nb_event_played"].max()
groupy.columns = ["Duration","Max_number_event"]
groupy.reset_index(inplace=True)
groupy.head(5)

,Season_Year,GameKey,PlayID,Duration,Max_number_event
0,2017,399,1336,205,6
1,2017,399,1776,862,10
2,2017,399,1884,971,8
3,2017,399,2572,212,6
4,2017,399,2712,663,11


In [6]:
select_player = pd.merge(groupe, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")
select_player.head(4)#The shape are checked to be sure of changes

,Season_Year,GameKey,PlayID,GSISID,Time_Player,Nb_event_played,Duration,Max_number_event
0,2017,399,1336,23449.0,29,1,205,6
1,2017,399,1336,23534.0,205,6,205,6
2,2017,399,1336,26035.0,205,6,205,6
3,2017,399,1336,27010.0,205,6,205,6


In [7]:
select_player = select_player.loc[(select_player.Time_Player == select_player.Duration )]
select_player.shape[0]

19758

In [13]:
test_22 = select_player.groupby(["Season_Year","GameKey","PlayID"])['GSISID'].count()
print(test_22.shape)
print(test_22.loc[test_22==22].shape)

(907,)
(853,)


We realize that there might be another way of detecting which player is involved in the action. 
Using the player role data csv


In [14]:
"""The events are a vital information, let's understand them better"""
set(df['Event'])

{nan,
 'penalty_accepted',
 'fumble',
 'pass_outcome_incomplete',
 'fair_catch',
 'line_set',
 'penalty_declined',
 'play_submit',
 'kick_received',
 'punt',
 'pass_outcome_caught',
 'lateral',
 'pass_arrived',
 'out_of_bounds',
 'punt_fake',
 'punt_received',
 'man_in_motion',
 'punt_play',
 'timeout_tv',
 'touchback',
 'shift',
 'huddle_break_offense',
 'fumble_defense_recovered',
 'tackle',
 'punt_land',
 'punt_muffed',
 'huddle_start_offense',
 'ball_snap',
 'first_contact',
 'touchdown',
 'run',
 'penalty_flag',
 'punt_downed',
 'punt_blocked',
 'timeout',
 'pass_forward',
 'snap_direct',
 'pass_outcome_interception',
 'fumble_offense_recovered'}

In [20]:
"""Let's analyse the plays and select only the punt plays to begin with"""
punt_play = pd.read_csv("./data/play_information.csv")
punt_play.head(2)

,Season_Year,Season_Type,GameKey,Game_Date,Week,PlayID,Game_Clock,YardLine,Quarter,Play_Type,Poss_Team,Home_Team_Visit_Team,Score_Home_Visiting,PlayDescription
0,2016,Pre,2,08/13/2016,2,191,12:30,LA 47,1,Punt,LA,LA-DAL,0 - 7,"(12:30) J.Hekker punts 52 yards to DAL 1, Cent..."
1,2016,Pre,2,08/13/2016,2,1132,12:08,LA 29,2,Punt,LA,LA-DAL,7 - 21,"(12:08) J.Hekker punts 51 yards to DAL 20, Cen..."


In [21]:
set(punt_play['Play_Type'])

{'Punt'}

It seems that all the plays in the info are puntplays.

In [22]:
simple_play_info = pd.DataFrame()
simple_play_info['Season_Year']=punt_play['Season_Year']
simple_play_info['GameKey'] = punt_play['GameKey']
simple_play_info['PlayID'] = punt_play['PlayID']
simple_play_info.head(5)


,Season_Year,GameKey,PlayID
0,2016,2,191
1,2016,2,1132
2,2016,2,1227
3,2016,2,1864
4,2016,2,2247


In [23]:
merged_plays = pd.merge(df, simple_play_info, on = ["Season_Year","GameKey","PlayID"], how = "inner")

In [24]:
print(merged_plays.shape)
df.shape

(9422296, 11)


(9433922, 11)

There is not a major difference between the two so we guess that the vaste majority of plays in the statistics dataframe are already punt plays.

In [25]:
new_merged_plays =  pd.merge(merged_plays, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")

In [26]:
new_merged_plays.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Duration,Max_number_event
0,2017,399,1336,23449.0,2017-09-08 01:31:11.000,41.560001,54.279999,0.13,78.879997,192.050003,NaN,205,6
1,2017,399,1336,23449.0,2017-09-08 01:31:11.100,41.540001,54.139999,0.14,77.760002,190.759995,NaN,205,6
2,2017,399,1336,23449.0,2017-09-08 01:31:11.200,41.509998,54.000000,0.14,76.459999,189.559998,NaN,205,6
3,2017,399,1336,23449.0,2017-09-08 01:31:11.300,41.490002,53.860001,0.14,75.800003,188.449997,NaN,205,6
4,2017,399,1336,23449.0,2017-09-08 01:31:11.400,41.470001,53.709999,0.15,75.769997,187.589996,NaN,205,6


In [27]:
"""Regarder s'il y a tjs un snapp ball à chaque partie"""


"Regarder s'il y a tjs un snapp ball à chaque partie"

In [18]:
"""On va s'attaquer aux infos players pour chaque joueurs."""
player_role_punt = pd.read_csv("./data/play_player_role_data.csv")
player_role_punt.head(4)

,Season_Year,GameKey,PlayID,GSISID,Role
0,2017,414,188,33704,PDL2
1,2017,414,1107,33704,PDL2
2,2017,424,1113,33704,PDR3
3,2017,424,1454,33704,PLR2


In [29]:
final_merge = pd.merge(new_merged_plays, player_role_punt, on = ["Season_Year","GameKey","PlayID", "GSISID"], how = "inner")
final_merge.head(3)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Duration,Max_number_event,Role
0,2017,399,1336,23534.0,2017-09-08 01:30:53.400,78.739998,29.820000,0.02,199.979996,168.630005,punt_play,205,6,P
1,2017,399,1336,23534.0,2017-09-08 01:30:53.500,78.750000,29.790001,0.03,199.009995,166.580002,NaN,205,6,P
2,2017,399,1336,23534.0,2017-09-08 01:30:53.600,78.769997,29.760000,0.04,198.029999,165.839996,NaN,205,6,P


In [32]:
print(new_merged_plays.shape)
final_merge.shape

(9422296, 13)


(7074107, 14)

In [31]:
final

(9422296, 13)